In [1]:
import tensorflow as tf
from keras.src.applications.resnet import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.resnet50 import preprocess_input


img_size = (224, 224)
batch_size = 32

train_ds = image_dataset_from_directory(
    'C:/Users/Natalia/Desktop/PADIA/endansys/dataset/training',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

val_ds = image_dataset_from_directory(
    'C:/Users/Natalia/Desktop/PADIA/endansys/dataset/validation',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_ds = image_dataset_from_directory(
    'C:/Users/Natalia/Desktop/PADIA/endansys/dataset/testing',
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(0.1, 0.1),
])

base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)
base_model.trainable = False

model = models.Sequential([
    data_augmentation,
    layers.Resizing(224, 224),
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(train_ds.class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(
    patience=10,
    restore_best_weights=True
)

model.fit(train_ds,
          validation_data=val_ds,
          epochs=35,
          callbacks=[early_stopping])

val_loss, val_acc = model.evaluate(val_ds)
print(f"Precisión en validación: {val_acc:.2%}")

test_loss, test_acc = model.evaluate(test_ds)
print(f"Precisión en test: {test_acc:.2%}")

#Aqui empieza el fine-tuning
base_model.trainable = True
fine_tune = 140

for layer in base_model.layers[:fine_tune]:
    layer.trainable = False

#Recompilar con un learning mas bajo para el fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(0.000001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Entrenamiento fine-tuning
model.fit(train_ds,
          validation_data=val_ds,
          epochs=25,
          callbacks=[early_stopping])

val_loss, val_acc = model.evaluate(val_ds)
print(f"Precisión en validación con fine-tuning: {val_acc:.2%}")

test_loss, test_acc = model.evaluate(test_ds)
print(f"Precisión en test con fine-tuning: {test_acc:.2%}")

Found 1429 files belonging to 56 classes.
Found 283 files belonging to 56 classes.
Found 367 files belonging to 56 classes.
Epoch 1/35



InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_12762]

In [3]:
class_names = train_ds.class_names

In [4]:
#print("Predicted index:", predicted_class)
print("Class names:", class_names)
print("Number of classes:", len(class_names))


Class names: ['Aguila_arpia', 'Anaconda', 'Armadillo_gigante', 'Boa_constrictor', 'Buho_cornudo', 'Cacique_candela', 'Caiman_aguja', 'Capibara', 'Caracara', 'Colibri_esmeralda', 'Condor_de_los_Andes', 'Cusumbo', 'Delfin_rosado', 'Flamenco', 'Gallito_de_roca', 'Garza_blanca', 'Guacamaya_azulamarilla', 'Guacamaya_roja', 'Guatin', 'Hormiguero_gigante', 'Iguana_verde', 'Jaguar', 'Jaguarundi', 'Lagarto_azul', 'Loro_orejiamarillo', 'Manati_del_Caribe', 'Marimonda', 'Martin_pescador', 'Mico_titi_gris', 'Mono_aullador', 'Mono_lanudo', 'Mono_nocturno', 'Musarana_de_cola_corta_colombiana', 'Neque', 'Nutria_gigante', 'Oso_andino', 'Oso_de_anteojos', 'Paujil', 'Pecari_de_collar', 'Perezoso_de_dos_dedos', 'Perico_carisucio', 'Perro_de_monte', 'Puma', 'Rana_de_cristal', 'Rana_dorada', 'Rana_venenosa_azul', 'Tapaculo', 'Tapir_amazonico', 'Tigrillo', 'Tinamu', 'Titi_cabeciblanco', 'Tortuga_charapa', 'Tucan_toco', 'Venado_cola_blanca', 'Zorrillo_listado', 'Zorro_cangrejero']
Number of classes: 56


In [5]:
import json

# Crear diccionario con los índices de clase
class_names = train_ds.class_names
class_indices = {name: idx for idx, name in enumerate(class_names)}

# Guardar en JSON
with open("class_indices.json", "w") as f:
    json.dump(class_indices, f)

print("class_indices.json")


class_indices.json


In [6]:
model.save("animal_species_classifier.h5")

In [7]:
import tensorflow as tf
model = tf.keras.models.load_model("animal_species_classifier.h5",
                                   custom_objects={'preprocess_input': preprocess_input})

In [10]:
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np

img_path = "C:/Users/Natalia/Desktop/PADIA/endansys/dataset/testing/Venado_cola_blanca/5.jpg"  # Cambia esto por la ruta de tu imagen
img_size = (224, 224)

# Cargar imagen y redimensionar
img = load_img(img_path, target_size=img_size)

# Convertir a array y escalar
img_array = img_to_array(img)
img_array = preprocess_input(img_array)  # Normalizar igual que en el modelo

# Expandir dimensiones para simular un batch de tamaño 1
img_array = np.expand_dims(img_array, axis=0)

predictions = model.predict(img_array)
class_names = train_ds.class_names

predicted_class = np.argmax(predictions[0])
confidence = 100 * np.max(predictions[0])

print(f"La imagen probablemente pertenece a '{class_names[predicted_class]}' con una confianza de {confidence:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
La imagen probablemente pertenece a 'Venado_cola_blanca' con una confianza de 94.66%
